<a href="https://colab.research.google.com/github/Davioliveira1305/PyTorch/blob/main/Toyota.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Descrição

**Uma grande concessionária de automóveis Toyota oferece aos compradores de carros Toyota novos a opção de comprar seu carro usado como parte de uma troca. Em particular, uma nova promoção promete pagar preços elevados pelos carros Toyota Corolla usados ​​para os compradores de um carro novo. O revendedor então vende o carro usado com um pequeno lucro. Para garantir um lucro razoável, o revendedor precisa ser capaz de prever o preço que receberá pelos carros usados. Para isso, foram coletados dados de todas as vendas anteriores de Toyota Corollas usados ​​na concessionária. Os dados incluem o preço de venda e outras informações sobre o carro, como idade, quilometragem, tipo de combustível e tamanho do motor.**

**Será feito uma rede neural para a predição dos valores de venda, utilizando o módulo PyTorch.**

## Importação das bibliotecas.

In [68]:
import torch
import pandas as pd
import numpy as np
from torch import nn
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

## Tratamento e Análise Exploratória dos Dados

In [69]:
data = pd.read_csv('/content/ToyotaCorolla.csv')

In [70]:
data

,Id,Model,Price,Age_08_04,Mfg_Month,Mfg_Year,KM,Fuel_Type,HP,Met_Color,...,Powered_Windows,Power_Steering,Radio,Mistlamps,Sport_Model,Backseat_Divider,Metallic_Rim,Radio_cassette,Parking_Assistant,Tow_Bar
0,1,TOYOTA Corolla 2.0 D4D HATCHB TERRA 2/3-Doors,13500,23,10,2002,46986,Diesel,90,1,...,1,1,0,0,0,1,0,0,0,0
1,2,TOYOTA Corolla 2.0 D4D HATCHB TERRA 2/3-Doors,13750,23,10,2002,72937,Diesel,90,1,...,0,1,0,0,0,1,0,0,0,0
2,3,TOYOTA Corolla 2.0 D4D HATCHB TERRA 2/3-Doors,13950,24,9,2002,41711,Diesel,90,1,...,0,1,0,0,0,1,0,0,0,0
3,4,TOYOTA Corolla 2.0 D4D HATCHB TERRA 2/3-Doors,14950,26,7,2002,48000,Diesel,90,0,...,0,1,0,0,0,1,0,0,0,0
4,5,TOYOTA Corolla 2.0 D4D HATCHB SOL 2/3-Doors,13750,30,3,2002,38500,Diesel,90,0,...,1,1,0,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1431,1438,TOYOTA Corolla 1.3 16V HATCHB G6 2/3-Doors,7500,69,12,1998,20544,Petrol,86,1,...,1,1,0,1,1,1,0,0,0,0
1432,1439,TOYOTA Corolla 1.3 16V HATCHB LINEA TERRA 2/3-...,10845,72,9,1998,19000,Petrol,86,0,...,0,1,0,0,1,1,0,0,0,0
1433,1440,TOYOTA Corolla 1.3 16V HATCHB LINEA TERRA 2/3-...,8500,71,10,1998,17016,Petrol,86,0,...,0,1,0,0,0,1,0,0,0,0
1434,1441,TOYOTA Corolla 1.3 16V HATCHB LINEA TERRA 2/3-...,7250,70,11,1998,16916,Petrol,86,1,...,0,0,0,0,0,1,0,0,0,0


**Domínio dos Atributos**

In [71]:
data.dtypes

Id                    int64
Model                object
Price                 int64
Age_08_04             int64
Mfg_Month             int64
Mfg_Year              int64
KM                    int64
Fuel_Type            object
HP                    int64
Met_Color             int64
Color                object
Automatic             int64
CC                    int64
Doors                 int64
Cylinders             int64
Gears                 int64
Quarterly_Tax         int64
Weight                int64
Mfr_Guarantee         int64
BOVAG_Guarantee       int64
Guarantee_Period      int64
ABS                   int64
Airbag_1              int64
Airbag_2              int64
Airco                 int64
Automatic_airco       int64
Boardcomputer         int64
CD_Player             int64
Central_Lock          int64
Powered_Windows       int64
Power_Steering        int64
Radio                 int64
Mistlamps             int64
Sport_Model           int64
Backseat_Divider      int64
Metallic_Rim        

In [72]:
data = data.drop('Model', axis='columns') # Nome do modelo do carro não é importante para a predição
data = data.drop('Color', axis = 'columns') # Cor do modelo não é importante para a predição
data['Price'] = pd.to_numeric(data['Price']) # Transformando o domínio da coluna Price para numérico
le = LabelEncoder()
data['Fuel_Type'] = le.fit_transform(data['Fuel_Type'])

**Verificando se existem valores nulos no dataset**

In [73]:
data.isnull().sum()

Id                   0
Price                0
Age_08_04            0
Mfg_Month            0
Mfg_Year             0
KM                   0
Fuel_Type            0
HP                   0
Met_Color            0
Automatic            0
CC                   0
Doors                0
Cylinders            0
Gears                0
Quarterly_Tax        0
Weight               0
Mfr_Guarantee        0
BOVAG_Guarantee      0
Guarantee_Period     0
ABS                  0
Airbag_1             0
Airbag_2             0
Airco                0
Automatic_airco      0
Boardcomputer        0
CD_Player            0
Central_Lock         0
Powered_Windows      0
Power_Steering       0
Radio                0
Mistlamps            0
Sport_Model          0
Backseat_Divider     0
Metallic_Rim         0
Radio_cassette       0
Parking_Assistant    0
Tow_Bar              0
dtype: int64

## Estrutura dos Dados

In [74]:
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

In [75]:
class MeuDataset(Dataset):
  def __init__(self,data ,label_column, train = True):
    self.data = data
    self.label_column = label_column

    # Dividir os dados em conjuntos de treinamento e teste
    train_indices, test_indices = train_test_split(range(len(self.data)), test_size=0.2, random_state=42)

    # Selecionar índices apropriados com base no modo (treinamento ou teste)
    if train:
        self.indices = train_indices
    else:
        self.indices = test_indices


  def __len__(self):
      return len(self.indices)

  def __getitem__(self, idx):
      sample = self.data.iloc[self.indices[idx]]

      # Extrair características (features) e rótulos (labels)
      features = sample.drop(self.label_column).values
      label = sample[self.label_column]

      return features, label



In [76]:
# Exemplo de uso para conjunto de treinamento
dataset_treino = MeuDataset(data, label_column='Price', train=True)

# Exemplo de uso para conjunto de teste
dataset_teste = MeuDataset(data, label_column='Price', train=False)

# Criar DataLoaders para ambos os conjuntos
dataloader_treino = DataLoader(dataset_treino, batch_size=30, shuffle=True)
dataloader_teste = DataLoader(dataset_teste, batch_size=30, shuffle=False)

In [83]:
data

,Id,Price,Age_08_04,Mfg_Month,Mfg_Year,KM,Fuel_Type,HP,Met_Color,Automatic,...,Powered_Windows,Power_Steering,Radio,Mistlamps,Sport_Model,Backseat_Divider,Metallic_Rim,Radio_cassette,Parking_Assistant,Tow_Bar
0,1,13500,23,10,2002,46986,1,90,1,0,...,1,1,0,0,0,1,0,0,0,0
1,2,13750,23,10,2002,72937,1,90,1,0,...,0,1,0,0,0,1,0,0,0,0
2,3,13950,24,9,2002,41711,1,90,1,0,...,0,1,0,0,0,1,0,0,0,0
3,4,14950,26,7,2002,48000,1,90,0,0,...,0,1,0,0,0,1,0,0,0,0
4,5,13750,30,3,2002,38500,1,90,0,0,...,1,1,0,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1431,1438,7500,69,12,1998,20544,2,86,1,0,...,1,1,0,1,1,1,0,0,0,0
1432,1439,10845,72,9,1998,19000,2,86,0,0,...,0,1,0,0,1,1,0,0,0,0
1433,1440,8500,71,10,1998,17016,2,86,0,0,...,0,1,0,0,0,1,0,0,0,0
1434,1441,7250,70,11,1998,16916,2,86,1,0,...,0,0,0,0,0,1,0,0,0,0


## Estrutura da Rede

In [ ]:
class Rede(nn.Module):
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(nn.Linear(36, 128),
                                nn.ReLU(),
                                nn.Linear(128, 64),
                                nn.Sigmoid(),
                                nn.Linear(64, 32),
                                nn.ReLU(),
                                nn.Linear(32, 16,),
                                nn.Sigmoid(),
                                nn.Linear(16, 1)
                                         )

  def forward(self, x)

